In [ ]:
pip install mysql-connector-python pandas streamlit google-api-python-client

In [ ]:
pip install selenium

In [ ]:
pip install webdriver-manager

In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains
import re
from selenium.common.exceptions import NoSuchElementException
import pandas as pd

In [ ]:
# Scrolling to the bottom to load all content
def scrolldown():
    scrolling = True
    while scrolling:
        old_page_source = driver.page_source
        body = driver.find_element(By.TAG_NAME, "body")
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(2)
        new_page_source = driver.page_source
        if new_page_source == old_page_source:
            scrolling = False

In [ ]:
#GETTING ROUTE NAMES AND ROUTE_LINKS

def route_name_link(link):

   driver = webdriver.Chrome()
   wait = WebDriverWait(driver, 10)
   driver.get(link)
   driver.implicitly_wait(3)    
   route_and_link=[]

   pc = wait.until(EC.presence_of_element_located((By.XPATH, "//*[@class='DC_117_paginationTable']")))
   num_pgs =len(driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs "))+1
   if num_pgs==1:
      #getting bus routes names
      Bus_routes=driver.find_elements(By.XPATH,"//a[@class='route']")
      allroute=[i.text for i in Bus_routes]
      
      #getting bus_routes_links
      nested_div = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.D117_main.D117_container')))
      route_links = nested_div.find_elements(By.CSS_SELECTOR, 'div.route_link')
      link=[i.find_element(By.TAG_NAME,"a").get_attribute('href') for i in route_links]
      
      r=zip(allroute,link)
      route_and_link.extend(r)
   else:
      for i in range(1,num_pgs):
         #getting bus routes names
         Bus_routes=driver.find_elements(By.XPATH,"//a[@class='route']")
         allroute=[i.text for i in Bus_routes]  
         
         #getting bus_routes_links
         nested_div = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.D117_main.D117_container')))
         route_links = nested_div.find_elements(By.CSS_SELECTOR, 'div.route_link')
         link=[i.find_element(By.TAG_NAME,"a").get_attribute('href') for i in route_links]
         
         r=zip(allroute,link)
         route_and_link.extend(r)
         
         if i==num_pgs-1:
            break
         else:
            next_page= pc.find_element(By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{i+ 1}"]')
            action = ActionChains(driver)
            action.move_to_element(next_page).perform()
            time.sleep(2)
            next_page.click()
            time.sleep(3)
      
   return route_and_link

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time

def route_name_link(link):
    # Initialize WebDriver
    driver = webdriver.Chrome()
    wait = WebDriverWait(driver, 10)
    driver.get(link)
    driver.implicitly_wait(3)
    route_and_link = []

    try:
        # Wait for the pagination table to load
        pagination_table = wait.until(
            EC.presence_of_element_located((By.XPATH, '//div[@class="DC_117_paginationTable"]'))
        )

        # Calculate the number of pages
        num_pgs = len(driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs")) + 1

        for page in range(1, num_pgs + 1):
            # Get bus route names
            bus_routes = driver.find_elements(By.XPATH, "//a[@class='route']")
            all_routes = [route.text for route in bus_routes]

            # Get bus route links
            nested_div = wait.until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'div.D117_main.D117_container'))
            )
            route_links = nested_div.find_elements(By.CSS_SELECTOR, 'div.route_link')
            links = [link.find_element(By.TAG_NAME, "a").get_attribute('href') for link in route_links]

            # Combine names and links
            route_and_link.extend(zip(all_routes, links))

            # Navigate to the next page if more pages exist
            if page < num_pgs:
                next_page = pagination_table.find_element(
                    By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page + 1}"]'
                )
                ActionChains(driver).move_to_element(next_page).perform()
                time.sleep(2)
                next_page.click()
                time.sleep(3)
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        # Close the WebDriver
        driver.quit()

    return route_and_link


In [ ]:
#click view buses
def click_view_buses():
    try:
        # Locate and click the 'View Buses' button if needed
            view_buses_button = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//div[@class='button']")))
            #print("len()", len(view_buses_button))
            for i in range(len(view_buses_button)-1,-1,-1):
                view_buses_button[i].click()
                time.sleep(2)
    except:
            pass
            for view_buses in view_buses_button:
                try:
                    view_buses.click()
                    scrolling = True
                    while scrolling:
                        old_page_source = driver.page_source
                        body = driver.find_element(By.TAG_NAME, "body")
                        body.send_keys(Keys.PAGE_DOWN)
                        time.sleep(2)
                        new_page_source = driver.page_source
                        if new_page_source == old_page_source:
                            scrolling = False
                    time.sleep(5)
                except Exception as e:
                    print(e)
    time.sleep(5)


In [ ]:
#getting bus details

def get_bus(link):
    driver.get(link)
    time.sleep(2)
    click_view_buses()
    #scrolldown()
    b=driver.find_elements(By.CSS_SELECTOR,"div[class='button']")
    for i in range(len(b)-1,-1,-1):
        try:
            b[i].click()
        except:
            print()
    x=0
    l=[]
    while True:
        x=x+1
        driver.execute_script('scrollBy(0,100)')
        #time.sleep(0.5)
        if(x>1000):
            break
    details = []
    bus_containers = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//div[contains(@class, 'clearfix bus-item')]")))
    for bus_container in bus_containers:
        try:
            
            #routes_name = bus_container.find_element(By.XPATH, ".//div[@class='D136_h1']").text
            busname = bus_container.find_element(By.XPATH, ".//div[@class='travels lh-24 f-bold d-color']").text
            bustype = bus_container.find_element(By.XPATH, ".//div[@class='bus-type f-12 m-top-16 l-color evBus']").text
            departuretiming = bus_container.find_element(By.XPATH, ".//div[@class='dp-time f-19 d-color f-bold']").text
            duration = bus_container.find_element(By.XPATH, ".//div[@class='dur l-color lh-24']").text
            reachtiming = bus_container.find_element(By.XPATH, ".//div[@class='bp-time f-19 d-color disp-Inline']").text
            rating = bus_container.find_element(By.XPATH, ".//div[@class='rating-sec lh-24']").text
            price = bus_container.find_element(By.XPATH, ".//div[@class='fare d-block']").text
            seat = bus_container.find_element(By.XPATH, ".//div[@class='column-eight w-15 fl']").text

            details.append({
                'links':link,
                #'Route_name': routes_name,
                'Bus_name': busname,
                'Bus_type': bustype,
                'Departure_time': departuretiming,
                'Duration': duration,
                'Arrival_time': reachtiming,
                'Ratings': rating,
                'Prices': price,
                'Seat_availability': seat,
            })
        except:
            pass     
    return details

In [ ]:
def flatten_list(nested_list):
    """Flattens a nested list."""
    return [item for sublist in nested_list for item in sublist]

In [ ]:
#to get all states data in Redbus
def all_state(state_link): 
    name_rlink= route_name_link(state_link)
    driver = webdriver.Chrome()
    route_link=[i[1] for i in name_rlink]

    all_data=[]
    for link in route_link:
        try:
            all_data.append(get_bus(link))
            driver = webdriver.Chrome()
        except:
            pass
            

# Flattening the collected data
    flattened_data = flatten_list(all_data)
    df=pd.DataFrame(flattened_data)
    return df

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Step 1: Initialize the WebDriver (e.g., for Chrome)
driver = webdriver.Chrome()

# Step 2: Open the webpage
driver.get('https://www.redbus.in/')  # Replace with your target URL

try:
    # Step 3: Wait for the element to be clickable
    element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, 'your_xpath_here')))  # Replace with your XPath
    element.click()
    print("Element clicked successfully.")
except Exception as e:
    print(f"Error: {e}")
finally:
    # Step 4: Close the browser
    driver.quit()

In [ ]:
# Example: Close a popup if it exists
try:
    popup_close_button = driver.find_element(By.XPATH, 'popup_close_button_xpath')
    popup_close_button.click()
except:
    print("No popup found or could not close it.")

# Now click the target element
element = driver.find_element(By.XPATH, 'your_xpath_here')
element.click()

In [ ]:
#main code

driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)
action = ActionChains(driver)

AP_data =all_state('https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile')
Kerala_data=all_state('https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile')
jammukashmir_data=all_state('https://www.redbus.in/online-booking/jksrtc')
punjab_data=all_state('https://www.redbus.in/online-booking/pepsu/?utm_source=rtchometile')
Rajasthan_data=all_state('https://www.redbus.in/online-booking/rsrtc/?utm_source=rtchometile')
Telangana_data=all_state('https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile')
West_bengal_data=all_state('https://www.redbus.in/online-booking/west-bengal-transport-corporation?utm_source=rtch')
karbi_anglong_data=all_state('https://www.redbus.in/online-booking/kaac-transport')
Kadamba_data= all_state('https://www.redbus.in/online-booking/ktcl/?utm_source=rtchometile')
Bihar_data=all_state('https://www.redbus.in/online-booking/bihar-state-road-transport-corporation-bsrtc/?utm_source=rtchometile')

#splitting route from links
AP_data_df = AP_data
AP_data_df['route'] = AP_data['links'].apply(lambda x: x.split('/')[-1])

Kerala_data_df = Kerala_data
Kerala_data_df['route'] = Kerala_data['links'].apply(lambda x: x.split('/')[-1])

jammukashmir_data_df=jammukashmir_data
jammukashmir_data_df['route'] = jammukashmir_data['links'].apply(lambda x: x.split('/')[-1])

punjab_data_df=punjab_data
punjab_data_df['route'] = punjab_data['links'].apply(lambda x: x.split('/')[-1])

Rajasthan_data_df=Rajasthan_data
Rajasthan_data_df['route'] = Rajasthan_data['links'].apply(lambda x: x.split('/')[-1])

Telangana_data_df=Telangana_data
Telangana_data_df['route'] = Telangana_data['links'].apply(lambda x: x.split('/')[-1])

West_bengal_data_df=West_bengal_data
West_bengal_data_df['route'] = West_bengal_data['links'].apply(lambda x: x.split('/')[-1])

karbi_anglong_data_df=karbi_anglong_data
karbi_anglong_data_df['route'] = karbi_anglong_data['links'].apply(lambda x: x.split('/')[-1])

Kadamba_data_df=Kadamba_data
Kadamba_data_df['route'] = Kadamba_data['links'].apply(lambda x: x.split('/')[-1])

Bihar_data_df=Bihar_data
Bihar_data_df['route'] = Bihar_data['links'].apply(lambda x: x.split('/')[-1])

In [ ]:
import pandas as pd
import os

# Define the directory path
directory = r'C:\Users\ADMIN\OneDrive\Desktop\OneDrive\Documents\Redbus'

# Create the directory if it doesn't exist
if not os.path.exists(directory):
    os.makedirs(directory)

# List of file names and corresponding data
csv_files = {
    'ap_data.csv': {'Column1': [1, 2, 3], 'Column2': ['A', 'B', 'C']},
    'Bihar_data.csv': {'Column1': [4, 5, 6], 'Column2': ['D', 'E', 'F']},
    'jammukashmir_data.csv': {'Column1': [7, 8, 9], 'Column2': ['G', 'H', 'I']},
    'Kadamba_data.csv': {'Column1': [10, 11, 12], 'Column2': ['J', 'K', 'L']},
    'Karbi_Anglong_data.csv': {'Column1': [13, 14, 15], 'Column2': ['M', 'N', 'O']},
    'Kerala_data.csv': {'Column1': [16, 17, 18], 'Column2': ['P', 'Q', 'R']},
    'Punjab_data.csv': {'Column1': [19, 20, 21], 'Column2': ['S', 'T', 'U']},
    'Rajasthan_data.csv': {'Column1': [22, 23, 24], 'Column2': ['V', 'W', 'X']},
    'Telangana_data.csv': {'Column1': [25, 26, 27], 'Column2': ['Y', 'Z', 'AA']},
    'westbengal_data.csv': {'Column1': [28, 29, 30], 'Column2': ['AB', 'AC', 'AD']}
}

# Loop through the dictionary and create CSV files
for file_name, data in csv_files.items():
    # Create a DataFrame
    df = pd.DataFrame(data)
    
    # Define the full file path
    file_path = os.path.join(directory, file_name)
    
    # Save the DataFrame as a CSV file
    df.to_csv(file_path, index=False)
    print(f"Created {file_path}")

print("All CSV files have been created successfully.")

In [ ]:
pip install pandas sqlalchemy pymysql

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# MySQL credentials
mysql_user = 'root'
mysql_password = '1234'
mysql_host = '127.0.0.1'
mysql_port = '3306'
mysql_database = 'redbus'

# Create a connection to MySQL database
engine = create_engine(f'mysql+pymysql://{mysql_user}:{mysql_password}@{mysql_host}:{mysql_port}/{mysql_database}', echo=True)

# List of CSV file paths
csv_files =[
    'C:\\Users\\ADMIN\\OneDrive\\Desktop\\OneDrive\\Documents\\Redbus\\ap_data.csv', 
    'C:\\Users\\ADMIN\\OneDrive\\Desktop\\OneDrive\\Documents\\Redbus\\Bihar_data.csv', 
    'C:\\Users\\ADMIN\\OneDrive\\Desktop\\OneDrive\\Documents\\Redbus\\jammukashmir_data.csv', 
    'C:\\Users\\ADMIN\\OneDrive\\Desktop\\OneDrive\\Documents\\Redbus\\Kadamba_data.csv', 
    'C:\\Users\\ADMIN\\OneDrive\\Desktop\\OneDrive\\Documents\\Redbus\\Karbi_Anglong_data.csv',
    'C:\\Users\\ADMIN\\OneDrive\\Desktop\\OneDrive\\Documents\\Redbus\\Kerala_data.csv', 
    'C:\\Users\\ADMIN\\OneDrive\\Desktop\\OneDrive\\Documents\\Redbus\\Punjab_data.csv',
    'C:\\Users\\ADMIN\\OneDrive\\Desktop\\OneDrive\\Documents\\Redbus\\Rajasthan_data.csv',
    'C:\\Users\\ADMIN\\OneDrive\\Desktop\\OneDrive\\Documents\\Redbus\\Telangana_data.csv',
    'C:\\Users\\ADMIN\\OneDrive\\Desktop\\OneDrive\\Documents\\Redbus\\westbengal_data.csv'
]

# List to hold DataFrames
dataframes = []

# Loop through the list of CSV files
for file in csv_files:
    try:
        # Attempt to read the CSV file
        df = pd.read_csv(file)
        # Append the DataFrame to the list if successful
        dataframes.append(df)
    except pd.errors.EmptyDataError:
        print(f"Warning: {file} is empty and has been skipped.")
    except Exception as e:
        print(f"Error: Could not read {file} due to {e}")

# Concatenate all valid DataFrames into a single DataFrame
combined_df = pd.concat(dataframes, ignore_index=True)

# Save the combined DataFrame to a new CSV file
combined_df.to_csv('combined_file.csv', index=False)

# Create a connection to MySQL database
engine = create_engine(f'mysql+pymysql://{mysql_user}:{mysql_password}@{mysql_host}:{mysql_port}/{mysql_database}', echo=True)

#load the datasets int sql
combined_df.to_sql('combined', con=engine, index=False, if_exists='replace')

In [ ]:
import streamlit as st
import pandas as pd
from sqlalchemy import create_engine

# Streamlit app
st.set_page_config(page_title="Redbus Data Scraping", layout="wide")
st.title(":bus: :orange[**Redbus Data Scraping with Selenium & Dynamic Filtering**]")

# Sidebar with project information
with st.sidebar: 
    st.markdown("### :green_book: :green[**About**]")
    st.markdown('''
    This project aims to streamline the collection and analysis of bus travel data.
    Using **:blue[Selenium]** for automated data extraction from Redbus, it gathers details like routes, schedules, prices, and seat availability.
    This initiative helps improve operational efficiency and strategic planning in the transportation industry.
    ''')
    # Additional notes or instructions
    st.markdown("### :memo: :green[**Additional Notes**]")
    st.markdown("For more details, please refer to the [Redbus website](https://www.redbus.in).")


# MySQL credentials
mysql_user = 'root'
mysql_password = '1234'
mysql_host = '127.0.0.1'
mysql_port = '3306'
mysql_database = 'redbus'

# Create a connection to the MySQL database
engine = create_engine(f'mysql+pymysql://{mysql_user}:{mysql_password}@{mysql_host}:{mysql_port}/{mysql_database}')

# Fetch data from the database
q1 = '''SELECT * FROM redbus_table'''
df_all = pd.read_sql(q1, engine)

# Main content layout
# st.markdown("### **:orange[Filter Options]**")
with st.container():
    col1, col2, col3 = st.columns(3)

    # Route filter 
    with col1:
        route_options = df_all['route'].unique()
        selected_route = st.selectbox('Select a Route:', route_options, index=None, placeholder='select a route')

    # Bus type filter
    q2 = '''SELECT * FROM redbus_table WHERE route = %s'''
    df_route = pd.read_sql(q2, engine, params=(selected_route,))
    with col2:
        bustype_options = df_route['Bus_type'].unique()
        selected_bustype = st.selectbox('Select a Bus Type:', bustype_options, index=None, placeholder='select a bustype')

    # Price filter
    q3 = '''SELECT * FROM redbus_table WHERE route = %s AND Bus_type = %s'''
    df_filtered = pd.read_sql(q3, engine, params=(selected_route, selected_bustype))
    with col3:
        price_options = df_filtered['Prices'].unique()
        selected_price = st.selectbox('Select a Price Range:', price_options, index=None, placeholder='select a price')

# Search button
if st.button('Search'):
    # Display filtered data
    st.markdown("### **:green[Filtered Bus Data]**")
    st.markdown(f"**Displaying data for:** *{selected_route}*, *{selected_bustype}*, *{selected_price}*")
    st.dataframe(df_filtered)

    # Download button for filtered data
    csv = df_filtered.to_csv(index=False)
    st.download_button(
        label="Download Filtered Data as CSV",
        data=csv,
        file_name='filtered_redbus_data.csv',
        mime='text/csv',
    )